### **LeafTracker Interactive Model**
This Jupyter notebook allows for experimentation with the LeafTracker model! The last Python module shows a demonstration of how the model works, and the other modules are there to prepare the model.

**Importing Libraries**

In [17]:
# import library dependencies, tested on python 3.9.6
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

**Loading the Model**<br>
I load the model from a set directory and print a summary to visualize how it works.

In [18]:
# load the pre-trained model from the appropriate file path
model = tf.keras.models.load_model('saved_model/my_model')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 180, 180, 3)       0         
                                                                 
 rescaling_1 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                  

**Model Variables**<br>
Here, I take key variables (for example, img_height, and batch_size) to make sure the image identification model can identify images. I also remake the training and validation datasets to find the classes.

In [19]:
# redefine values from the model
data_dir = "tpc-imgs/"
img_height = img_width = 180
batch_size = 32

# remake the training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# remake the validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# find the class names
class_names = train_ds.class_names



**Image Identification**<br>
Here, the model takes in an image URL and identifies the top three likeliest possibilities. Feel free to tweak the URL to determine the model's accuracy!

In [20]:
# take an image to use for testing
# feel free to switch the url for experimentation!
# url = "https://mbgna.umich.edu/wp-content/uploads/2021/03/JITP-berries-Chesapeake-Conservation-Landing.jpg"
url = "http://extension.msstate.edu/sites/default/files/publication-images/IS0642/poisonivy1.jpg"
# url = "https://extension.illinois.edu/photolib/lib17//inset/shrubs105a.jpg"

path = tf.keras.utils.get_file(origin=url)

# load the image into a variable
img = tf.keras.utils.load_img(
    path, target_size=(img_height, img_width)
)

# convert the image into a tensor and create a batch for testing
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

# find the top three likeliest plants based on probabilities
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])
top_three = np.array(score).argsort()[-3:][::-1]
numpy_array = score.numpy()

# convert the folder names into English words then print the three likeliest probabilities
for i in top_three:
    words = class_names[i].split("_")
    name = " ".join([word.capitalize() for word in words])
    print(
        "This image likely belongs to {} with {:.2f}% confidence."
        .format(name, 100 * numpy_array[i])
    )

1/1 [==============================] - 0s 150ms/step
This image likely belongs to Poison Sumac with 52.95% confidence.
This image likely belongs to Eastern Poison Ivy with 40.35% confidence.
This image likely belongs to Western Poison Oak with 4.41% confidence.
